### Create Figure 1a - THOR bombing overview
Creates the overview map of THOR bombing missions across Southeast Asia.

#### Input:
* *thor_path*: THOR bombing data (csv file)
* *vnm_gadm_path*: GADM country outlines for Vietnam (geojson file)
* *khm_gadm_path*: GADM country outlines for Cambodia (geojson file)
* *lao_gadm_path*: GADM country outlines for Lao PDR (geojson file)
* *footprint_path_sa*: KH-9 study area footprints (geojson files)


#### Parameters:
* *fontsize*: Fontsize of the figure  
* *thor_color*: Color used for visualising the THOR bombing targets
* *dpi*: DPI to use when saving the figure

#### Outputs
* *plots_folder*: Folder to save the figure in

#### Created paper content:
* **Figure 1a**: Main parts of Figure 1a, text elements were added later in Powerpoint

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import matplotlib.pyplot as plt

from utils import create_dir, load_config
from analysis import load_thor
from matplotlib.lines import Line2D

In [2]:
config = load_config("../config.yaml")

### Load data

In [3]:
vietnam = gpd.read_file(config.get("vnm_gadm_path"))
vietnam = cfeature.ShapelyFeature(vietnam.geometry, crs=ccrs.PlateCarree())
cambodia = gpd.read_file(config.get("khm_gadm_path"))
cambodia = cfeature.ShapelyFeature(cambodia.geometry, crs=ccrs.PlateCarree())
lao = gpd.read_file(config.get("lao_gadm_path"))
lao = cfeature.ShapelyFeature(lao.geometry, crs=ccrs.PlateCarree())

In [4]:
cities = {
    "\n\n\nHo Chi \n Minh City": [10.762622, 106.660172]
}

capitals = {
    "\n\nHanoi": [21.028511, 105.804817],
    "\n\nPhnom Penh": [11.562108, 104.888535],
    "\n\nVientiane": [17.974855, 102.630867]
}

countries_plot = {
    "LAOS": [19.680, 102.089],
    "VIETNAM": [22.028, 104.938],
    "CAMBODIA": [12.764, 104.366]
}

In [5]:
qt_footprint = gpd.read_file(config.get("footprint_path_sa").format(study_area="quang_tri"))
qt_footprint = cfeature.ShapelyFeature(qt_footprint.geometry, crs=ccrs.UTM(48))
tba_footprint =  gpd.read_file(config.get("footprint_path_sa").format(study_area="tri_border_area"))
tba_footprint = cfeature.ShapelyFeature(tba_footprint.geometry, crs=ccrs.UTM(48))

In [6]:
# load THOR data
thor = load_thor(config.get("thor_path"))
# subset to kinetic missions
thor_kinetic = thor[(thor["mfunc_desc_class"] == "KINETIC")]

### Figure 1a

In [7]:
# key parameters for the plot
fontsize = config.get("fontsize")
thor_color = config.get("thor_color")
proj = ccrs.UTM(48)
dpi = config.get("dpi")
figure_size = 14.5 # figure size in cm
cm = 1/2.54 # conversion factor between inches and cm
save_params = {"format": "png", "dpi": dpi, "transparent": False, "bbox_inches": "tight", "pad_inches": 0}

output_folder = config.get("plots_folder")
create_dir(output_folder)

Directory already exists: ../outputs/plots


In [8]:
fig = plt.figure(figsize=(figure_size*cm, figure_size*cm), dpi=dpi)
ax = plt.axes(projection=proj)
ax.set_extent([100.04458465500008, 109.56917000000007, 8.281355000000099, 23.49269256700004], crs=ccrs.PlateCarree())

ax.add_feature(vietnam, facecolor="none", edgecolor="black", zorder=6)
ax.add_feature(cambodia, facecolor="none", edgecolor="black", zorder=6)
ax.add_feature(lao, facecolor="none", edgecolor="black", zorder=6)

ax.add_feature(qt_footprint, facecolor="none", edgecolor="darkblue", linewidth=1.5, zorder=7)
ax.add_feature(tba_footprint, facecolor="none", edgecolor="darkblue", linewidth=1.5, zorder=7)

for name, coords in capitals.items():
    ax.scatter(x=coords[1], y=coords[0], marker="D", s=10, color="black", transform=ccrs.PlateCarree(), zorder=6)

for name, coords in cities.items():
    ax.scatter(x=coords[1], y=coords[0], marker="o", s=10, color="black", transform=ccrs.PlateCarree(), zorder=6)
    
ax.axis("off")

ax.scatter(thor_kinetic["tgtlonddd_ddd_wgs84"], thor_kinetic["tgtlatdd_ddd_wgs84"],
           marker=".", 
           color=thor_color,
           edgecolor="none",
           alpha=0.2,
           s=0.1, 
           transform=ccrs.PlateCarree(),
           zorder=5)

# add legend
legend_handles = [Line2D([0], [0], marker=".", color="black", label="", lw=0,
      markerfacecolor=thor_color, markeredgecolor="none", markersize=8)]
ax.legend(handles=legend_handles, loc="center left", labelcolor="black", prop={"weight": "bold", "size": fontsize}, 
          facecolor="white", edgecolor="white")

plt.savefig(f"{output_folder}/figure_1a_vietnam_overview.png", **save_params)
plt.close()